# Assistive Technology (2023-03-24)

## Email System

`Install-Package MailKit`

In [23]:
#r "nuget:MailKit"
#r "nuget:DotNetEnv"

// Load .env file
DotNetEnv.Env.TraversePath().Load();

Installed Packages DotNetEnv, 2.5.0 MailKit, 3.6.0

## Receiving Email

In [24]:
using MimeKit;
using MailKit;
using MailKit.Search;
using MailKit.Net.Imap;

using (var client = new ImapClient ()) {
    client.Connect ("imap.gmail.com", 993, true);

    client.Authenticate (DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    // The Inbox folder is always available on all IMAP servers...
    var inbox = client.Inbox;
    inbox.Open (FolderAccess.ReadOnly);

    Console.WriteLine ("Total messages: {0}", inbox.Count);
    Console.WriteLine ("Recent messages: {0}", inbox.Recent);

    for (int i = 0; i < inbox.Count; i++) {
        var message = inbox.GetMessage (i);
        Console.WriteLine ("Subject: {0}", message.Subject);
    }

    client.Disconnect (true);
}

Total messages: 5
Recent messages: 0
Subject: Your Google Account is live – now help your business grow
Subject: Recovery email verified for your Google Account
Subject: 2-Step Verification turned on
Subject: Security alert
Subject: Hello World


## Sending Mail

In [ ]:
using MailKit.Net.Smtp;
using MailKit;
using MimeKit;

var message = new MimeMessage();
message.From.Add(new MailboxAddress("Sender Name", DotNetEnv.Env.GetString("EMAIL__ACCOUNT") + "@gmail.com"));
message.To.Add(new MailboxAddress("Receiver Name", DotNetEnv.Env.GetString("EMAIL__ACCOUNT") + "@gmail.com"));
message.Subject = "Sample Subject Line";

message.Body = new TextPart("plain") {
    Text = @"Lorem ipsum dolor sit amet, consectetur adipiscing elit, 
    sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
    Scelerisque felis imperdiet proin fermentum leo vel orci. 
    Mattis molestie a iaculis at erat pellentesque adipiscing commodo. 
    Urna porttitor rhoncus dolor purus non enim. Amet nisl purus in mollis nunc sed id semper."
};

using (var client = new SmtpClient()) {
    client.Connect("smtp.gmail.com", 587);

    client.AuthenticationMechanisms.Remove("XOAUTH2");

    // Note: only needed if the SMTP server requires authentication.
    client.Authenticate(DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    client.Send(message);
    client.Disconnect(true);
}